# Colour classifying neural network classification regions
This is an interactive Jupyter Notebook for showing the colour classification distributions of my colour classifying neural network.

Importing all the things that we need, also setting the filename for the weights file.

In [7]:
from main import *
from ipywidgets import interact, interactive, fixed, interact_manual
from skimage import measure
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

FILENAME = "weights.npz"

Importing the weights from the file.

In [8]:
hl_weights, ol_weights = import_weights(FILENAME)

Creating an interactive window to show a 2D representation of two of the RGB values changing as one is held constant, classifying those colours, and drawing separation lines between different colours. This code is kind of slow right now, I would like to find a way to make it faster.

In [9]:
@interact(slider=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5), 
          slider_colour=widgets.RadioButtons(options=['Red', 'Green', 'Blue'],description='Select colour to change with slider:'))
def update_canvas(slider, slider_colour):
    selected_colour = slider_colour
    # Create a canvas with specified dimensions
    canvas_width = 100
    canvas_height = 100

    # Create a canvas with the gradient
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

    for y in range(canvas_height):
        for x in range(canvas_width):
            # Interpolate the color at each pixel
            colour1 = x/canvas_width*255
            colour2 = y/canvas_height*255
            colour3 = slider*255
            
            if selected_colour == "Blue":
                canvas[y, x] = [colour1, colour2, colour3]
            elif selected_colour == "Green":
                canvas[y, x] = [colour1, colour3, colour2]
            elif selected_colour == "Red":
                canvas[y, x] = [colour3, colour1, colour2]
                
    classifications = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
    for y in range(canvas_height):
        for x in range(canvas_width):
            current_colour = [c/255 for c in canvas[y, x]]
            classifications[y, x] = np.argmax(forward_pass(np.array(current_colour), hl_weights, ol_weights))
    
    for i in range(0, len(COLOUR_NAMES)):
        contours = measure.find_contours(classifications, i)
        for contour in contours:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=1, color=COLOUR_NAMES[i])
        
        x_indices, y_indices = np.where(classifications == i)
        centroid_x = np.mean(x_indices)
        centroid_y = np.mean(y_indices)
        plt.text(centroid_y-10, centroid_x, COLOUR_NAMES[i], fontsize=12, color="black")

    # Display the canvas with the color gradient
    plt.imshow(canvas)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='slider', max=1.0), RadioButtons(description='Select …